# Data Analytics Group Project - Predicting Happiness Score

In this notebook we try to train a model that will attempt to correctly predict Happinesss Score. We have a target feature (Happiness Score) and a whole range of descripitve features, some which came with the Happiness Score data file, others we had to collect ourselves from a variety of different places such as the WHO. 



In [83]:
# Import all necessary packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
import statsmodels.formula.api as sm
from sklearn import metrics
import seaborn as sns

# Plots appear in notebook
%matplotlib inline

#Turn on automcompletion
%config IPCompleter.greedy=True

## Part 1: Combining Data from different Data Sets
### Part A - Load main dataframe

The main bulk of the data stems directly from the world happiness report website at http://worldhappiness.report/". There we obtained a file containing the happiness scores from 2008-2017. It actually goes back to 2006 for some countries, however we decided to just focus on the last 10 years. In addition it contains a couple of other features which we thought are very well suited as descriptive features. Some, where we were not quite sure on their meaning and quality we decided to drop and others were simply not of interest. 

For more detailed information on each descriptive feature, there is a pdf in the docs folder from the world happiness report,  which outlines exactly how these features were obtained. 

In [ ]:
# Load in the csv from the happiness report website
main_df = pd.read_csv("raw_data/happiness&descriptivefeat_2008-2017.csv")
main_df.head()

In [ ]:
# Rename target feature "life ladder" to "happiness Score"
main_df = main_df.rename(columns={'Life Ladder': 'Happiness Score'})

In [ ]:
# Change Happiness Score into categorical feature (0-10)
main_df["Happiness Score"] = round(main_df["Happiness Score"])

In [ ]:
# Drop unwanted columns and unnecessary columns
main_df.drop(['Standard deviation of ladder by country-year', 'Standard deviation/Mean of ladder by country-year', 'GINI index (World Bank estimate)', 'GINI index (World Bank estimate), average 2000-15', 'gini of household income reported in Gallup, by wp5-year'], axis=1, inplace=True )
main_df.head()

### Part B - Add GDP (replacing log GDP)
There were a number of other datasets which we thought were interesting and maybe relevant in predicting happiness. One of them was crime rate. Unfortunately, the dataset only ranged back to 2012, and covered a different amount of countries in each year, and never the full amount of our main dataset which covers 162 countries starting from 2008-2017. So we decided to not add crime rate as it would simply add too much missing data

However, we did add some other features. One of them was raw GDP, which we used to replace the "Log GDP per capita". 

In [ ]:
# read in GDP data
raw_gdp_df = pd.read_csv("raw_data/GDP2008_to_2017.csv")
raw_gdp_df.head()

In [ ]:
# drop unnecessary columns
raw_gdp_df.drop(["Units", "Scale", "Country/Series-specific Notes", "Estimates Start After"], axis=1, inplace=True)
raw_gdp_df.head()

In [ ]:
# Check for duplicated rows
print("Number of duplicated rows: {}".format(sum(raw_gdp_df.duplicated())))

In [ ]:
# Convert raw_gdp_df into one more suitable for merging
raw_gdp_df = raw_gdp_df.rename(columns={"Country": "country"})
raw_gdp_df = pd.melt(raw_gdp_df, id_vars=["country"], var_name="year", value_name="GDP").sort_values(["country", "year"])
raw_gdp_df.head()

In [ ]:
# convert dataype of Year to int64
raw_gdp_df.year = raw_gdp_df.year.astype("int64", inplace=True)

In [ ]:
# merge with the main df based on year and country
main_df = pd.merge(main_df, raw_gdp_df,  how='left', left_on=['country', "year"], right_on = ['country', "year"])
main_df.drop(["Log GDP per capita"], axis=1, inplace=True)
main_df.head()

### Part C - Add life expectancy from WHO
Next we added additional life expectancy data from WHO besides the already existing "Healthy life expectancy at birth" feature

In [ ]:
# read in WHO data
who_df = pd.read_csv("raw_data/WHOLifeExpectancy.csv")

# select desired columns
who_df = who_df[["Unnamed: 0", "Unnamed: 1", "Life expectancy at birth (years)", "Life expectancy at age 60 (years)"]]

# rename columns
who_df = who_df.rename(columns={"Unnamed: 0": "country", "Unnamed: 1": "year", "Life expectancy at birth (years)": "Life expectancy birth", "Life expectancy at age 60 (years)": "Life expectancy age 60"})

# drop first row (contains useless data)
who_df.drop(who_df.index[0], inplace=True)

# change dtypes
who_df.year = who_df.year.astype("int64", inplace=True)
who_df["Life expectancy birth"] = who_df["Life expectancy birth"].astype(float, inplace=True)
who_df["Life expectancy age 60"] = who_df["Life expectancy age 60"].astype(float, inplace=True)

who_df.head()

In [ ]:
# merge with the main df based on year and country
main_df = pd.merge(main_df, who_df,  how='left', left_on=['country', "year"], right_on = ['country', "year"])
main_df.head()

### Part D - Add Infant Mortality Rate

In [ ]:
# read in infant mortality data
im_df = pd.read_csv("raw_data/Dying_baby_data.csv")

# rename columns
im_df = im_df.rename(columns={"Country": "country", "Year": "year", "Infant mortality rate (probability of dying between birth and age 1 per 1000 live births)": "Infant mortality rate", "Neonatal mortality rate (per 1000 live births)": "Neonatal mortality rate", "Under-five mortality rate (probability of dying by age 5 per 1000 live births)": "Under-five mortality rate"})

im_df.head()

In [ ]:
# only keep number of deaths for mortality rate
im_df[['Infant mortality rate', 'Neonatal mortality rate', 'Under-five mortality rate']] = im_df[[
    'Infant mortality rate', 'Neonatal mortality rate', 'Under-five mortality rate']].apply(lambda row: row.str.replace("(\[.+\])", ''))
im_df.head()

In [ ]:
# merge with the main df based on year and country
main_df = pd.merge(main_df, im_df,  how='left', left_on=['country', "year"], right_on = ['country', "year"])
main_df.head()

### Part E - Add CPI (world corruption ranking)

In [ ]:
# read in CPI mortality data
cpi_df = pd.read_csv("raw_data/transparency_international_CPI.csv")

# drop unwanted columns
cpi_df.drop(["ISO3"], axis=1, inplace=True)

# rename columns
cpi_df = cpi_df.rename(columns={"Country": "country", "CPI score 2017": "2017", "CPI score 2016": "2016", "CPI score 2015": "2015", "CPI score 2014": "2014", "CPI Score 2013": "2013", "CPI Score 2012": "2012"})

# convert into a format suitable for merging
cpi_df = pd.melt(cpi_df, id_vars=["country"], var_name="year", value_name="CPI").sort_values(["country", "year"])

# conver dataype of Year to int64
cpi_df.year = cpi_df.year.astype("int64", inplace=True)
cpi_df.head()                       

In [ ]:
# merge with the main df based on year and country
main_df = pd.merge(main_df, cpi_df,  how='left', left_on=['country', "year"], right_on = ['country', "year"])
main_df.head()

### Part F - Write merged file to csv

In [ ]:
main_df.to_csv("processed_data/merged_happiness_data.csv", index=False)

## Part 2: Data Quality Report
### Part A - Overview & Analysis

In [ ]:
# load in the merged data
df = pd.read_csv("processed_data/merged_happiness_data.csv")
df.head()

#### Target Feature:
The target feature is of course Happiness Score which is the National average response to the question “Please imagine a ladder, with steps numbered from 0 at the bottom to 10 at the top. The top of the ladder represents the best possible life for you and the bottom of the ladder represents the worst possible life for you. On which step of the ladder would you say you personally feel you stand at this time?”

#### Descriptive Features
The descriptive features are as follows:
1. **country** 
2. **year**
3. **Social support** - The national average of the binary responses (either 0 or 1) to the question “If you were in trouble, do you have relatives or friends you can count on to help you whenever you need them, or not?”
4. **Healthy life expectancy at birth** - Life expectancy spent in “Good Health”. So not just Life Expectancy. Naturally Smaller than LE. 
5. **Freedom to make life choices** - The national average of responses to the question “Are you satisfied or dissatisfied with your freedom to choose what you do with your life?”
6. **Generosity** - The residual of regressing national average of response to the question “Have you donated money to a charity in the past month?” on GDP per capita.
7. **Perceptions of corruption** - The measure is the national average of the survey responses to two questions: (I) “Is corruption widespread throughout the government or not” and (II) “Is corruption widespread within businesses or not?”
8. **Positive affect** - The average of three positive affect measures: happiness, laugh and enjoyment. These measures are computed from responses to question such as “Did you smile or laugh a lot yesterday?”.
9. **Negative affect** - Defined as the average of three negative affect measures. They are worry, sadness and anger, respectively. These measures are computed from responses to question such as “Did you feel sad a lot yesterday?”.
10. **Confidence in national government** - Answer to “Do you have confidence in each of the following, or not? How about the national government?”
11. **Democratic Quality & Delivery Quality**: both are based on Worldwide Governance Indicators (WGI) project. The original data has six dimensions: Voice and Accountability, Political Stability and Absence of Violence, GovernmentEffectiveness, Regulatory Quality, Rule of Law, Control of Corruption which are reduced to two here.
12. **GDP**
13. **Life expectancy birth**
14. **Life expectancy age 60**
15. **Infant mortality rate:** Deaths per 1000 births
16. **Neonatal mortality rate:** neonatal deaths per 1000 births
17. **Under-five mortality rate:** under five deaths per 1000 births
18. **CPI:** World Corruption ranking


In [ ]:
# Check for duplicated rows
print("Number of duplicated rows: {}".format(sum(main_df.duplicated())))

In [ ]:
# Check dtypes
df.dtypes

In [ ]:
# Convert country and Happiness Score to categorical
df["country"] = df["country"].astype("category")
df["Happiness Score"] = df["Happiness Score"].astype("category")
df["year"] = df["year"].astype(int)

In [ ]:
# Check number of countries
print("Number of countries in dataset: {}".format(len(main_df.country.unique())))

In [ ]:
# Show for how many countries the happiness score is available in what time span
ax = df.country.value_counts().value_counts().plot(kind="bar")
ax.set_xlabel("Recorded Years of Happiness Score")
ax.set_ylabel("Number of Countries")

As we can see there are 64 countries for which we have a happiness score 12 years back. We are however only interested in the first 10 years, so we drop anything that is before 2008. Other countries have less data available but we thought this was no problem.

In [ ]:
# drop years 2006 and 2007
df = df.drop(df[df.year < 2008].index)

In [ ]:
# Show for how many countries the happiness score is available in what time span
ax = df.country.value_counts().value_counts().plot(kind="bar")
ax.set_xlabel("Recorded Years of Happiness Score")
ax.set_ylabel("Number of Countries")

### Part B - Data Quality Report
#### Continous Data

In [ ]:
#Get descriptive stats for continous data
con_feat = df.describe(exclude=["category"]).T

#Create a DF containing only continous data
con_df = df.select_dtypes(include=[np.number])
con_df.index = df.index

#Add Median
con_feat["median"] = con_df.median()

#Add Cardinality
con_feat["card"] = con_df.apply(pd.Series.nunique)

#Add percentage missing by checking for missing data in each column. 
#NOTE: This only considers NA values as missing. Data could still be absent
#      but replaced with a "dummy" entry such as a blank space or a zero.
#      This issue Will be adressed in the Data Quality plan.
con_feat["missPerc"] = con_df.isnull().apply(pd.Series.sum)/df.shape[0]

In [ ]:
#Prettify the result, rearrange columns and get rid of the 50th percentile column (same as median).
con_feat = con_feat[["count", "missPerc", "card", "min", "25%", "mean", "median", "75%", "max", "std"]]
print("Descriptive statistics for continous features:\n")
#Rounding will not get saved. Is only done so that an overview of the df is easier to read.
display(con_feat.round(2))

In [ ]:
#Plot histogram for continous features
df.hist(figsize=(15,20))

#Show the plot and then proceed to clear the current figure to reuse for other plots
plt.show()
plt.clf()

#### Categorical Data

In [ ]:
#Get descriptive stats for continous data
cat_feat = df.describe(include=["category"]).T

#rename some coulmns for improved understanding
cat_feat.rename(index=str, columns={"unique":"card", "top":"mode", "freq":"modeFreq"}, inplace=True)

#Create a DF containing only categorical data
cat_df = df.select_dtypes(include=["category"])
cat_df.index = df.index

#Add percentage missing by checking for missing data in each column. 
#NOTE: This only considers NA values as missing. Data could still be absent
#      but replaced with a "dummy" entry such as a blank space or a zero.
#      This issue Will be adressed in the Data Quality plan.
missing = cat_df.isnull().apply(pd.Series.sum)
print("Missing count for each categorical feature:\n")
print(missing)
cat_feat["missPerc"] = missing/1000 * 100

In [ ]:
#Add mode percentage 
cat_feat["modePerc"] = cat_feat.modeFreq/1000 * 100

#Add 2n mode category, frequency and percentage
#First get the 2n mode value for each column and its count
sec_mode = []
sec_mode_count = []
for col in cat_df:
    cat_df[col].cat.categories
    sec_mode.append(cat_df[col].value_counts().index[1])
    sec_mode_count.append(cat_df[col].value_counts().iloc[1])

#Add to df
cat_feat["secMode"] = sec_mode
cat_feat["secModeFreq"] = sec_mode_count
cat_feat["secModePerc"] = cat_feat.secModeFreq/1000 * 100

#Prettify the result, rearrange columns etc.
cat_feat = cat_feat[["count", "missPerc", "card", "mode", "modeFreq", "modePerc", "secMode", "secModeFreq", "secModePerc" ]]
print("Descriptive statistics for categorical features:")
display(cat_feat)

In [ ]:
#plot happiness
fig = cat_df["Happiness Score"].value_counts().plot(kind="bar", title=col, figsize=(15, 10))
plt.show()
    
#Close pdf and clear figure
plt.clf()

### Part C - Data Quality Plan
The categorical data seems fine. We have no missing values in any row and the cardinality seems appropriate for both features. There seems to be more work for the continous data as we have a substantial amount of missing data. Before that though a quick look at the other issues. GDP seemed to have a very low minimum and very high maximum. So I investigate it next.

In [ ]:
df.dtypes

In [ ]:
# Min GDP Country
df[df.GDP == min(df.GDP)]

In [ ]:
# Max GDP country
df[df.GDP == max(df.GDP)]

The values for GDP seem to be fine after seeing that the lowest one belongs to a very small country and the highest one to USA. Next I will investigate any missing data. There are several columns for which only a tiny fraction is missing such as Social support shown below. So for these cases I just replace each missing value with the countries average value for that column

In [ ]:
df[df["Social support"].isnull()]

In [ ]:
# Define a helper function which gets the average for a selected country's column values
def get_average(Country, col):
    '''Gets the average of a country's column values
    '''
    average = df[df.country==Country][col].mean()
    return average

In [ ]:
# Replace the missing values for "Social support", "Healthy life expectancy at birth", "Freedom to make life choices", "Generosity", "Perceptions of corruption", "Positive affect", "Negative affect", "Confidence in national government", "Democratic Quality", "Delivery Quality", "GDP"
for col in ["Social support", "Healthy life expectancy at birth", "Freedom to make life choices", "Generosity", "Perceptions of corruption", "Positive affect", "Negative affect", "Confidence in national government", "Democratic Quality", "Delivery Quality", "GDP"]:
    Countries = df[df[col].isnull()].country
    Years =  df[df[col].isnull()].year
    for Country, Year in zip(Countries, Years):
        df.at[(df.country==Country) & (df.year==Year), col] =  get_average(Country, col)

Unfortunately this does not work for all of the selected columns. Some countires have no value at all for a certain feature, so naturally there is no average value that can be used to replace the missing value. One could replace the missing value by the worldwide average which is what I will do

In [ ]:
for col in ["Healthy life expectancy at birth", "Generosity", "Perceptions of corruption", "Confidence in national government", "Democratic Quality", "Delivery Quality", "GDP"]:
    Countries = df[df[col].isnull()].country
    Years =  df[df[col].isnull()].year
    for Country, Year in zip(Countries, Years):
        df.at[(df.country==Country) & (df.year==Year), col] =  df[col].mean()

There are however a couple of features for which more data is missing, although there is an observanle trend to the amount that is missing. This depends on the dataset where the features came from. Data from the WHO has 33% missing, Infant mortality has 23% missing and CPI has 41% missing. Looking for trends in the missing data:

In [ ]:
df[df["Life expectancy birth"].isnull()].year.value_counts()

In [ ]:
df[df["Life expectancy age 60"].isnull()].year.value_counts()

In [ ]:
# same for the other two values that came with the infant mortality data set
df[df["Infant mortality rate"].isnull()].year.value_counts()

In [ ]:
df[df["CPI"].isnull()].year.value_counts()

Data seems to be missing mostly for the years 2016 and 2017. This makes sense as these are very recent and therefore not much was published on it. However since it is under 30% for all except CPI, I replace it with the countrys average, and if that does not exist then with the worldwide average. Hover CPI will be dropped as we do not have enough data considering it only ranges back to 2012. This is unfortunate, and if one wanted to include it in the dataset one would have to focus on only 2012-2017. Replacing five years worth of data with an average seems unrealistic to me. For one or two years I imagine that the average is a good approximation but for CPI there is just too much missing

In [ ]:
# Replace the missing values for "Life expectancy birth", "Life expectancy age 60", "Infant mortality rate", "Neonatal mortality rate", "Under-five mortality rate"
for col in ["Life expectancy birth", "Life expectancy age 60", "Infant mortality rate", "Neonatal mortality rate", "Under-five mortality rate"]:
    Countries = df[df[col].isnull()].country
    Years =  df[df[col].isnull()].year
    for Country, Year in zip(Countries, Years):
        df.at[(df.country==Country) & (df.year==Year), col] =  get_average(Country, col)

In [ ]:
# Replace by worldwide average evrything for which there was no national average
for col in ["Life expectancy birth", "Life expectancy age 60", "Infant mortality rate", "Neonatal mortality rate", "Under-five mortality rate"]:
    Countries = df[df[col].isnull()].country
    Years =  df[df[col].isnull()].year
    for Country, Year in zip(Countries, Years):
        df.at[(df.country==Country) & (df.year==Year), col] =  df[col].mean()

In [ ]:
# drop CPI
df.drop(["CPI"], axis=1, inplace=True)

In [ ]:
# write cleaned df to csv
df.to_csv("processed_data/cleaned_merged_happiness_data.csv", index=False)

## PART 3: Data Understanding
The last section in this notebook will be comitted to Data Understanding. This does not look for correlation between target and descriptive features but rather serves as an overview of the data and how it has developed over the past years. Maybe we will observe some interesting trends which can then be confirmed via the correlation steps in the next part.  

In [ ]:
# Convert the target feature from categorical to numeric for plotting
df["Happiness Score"] = df["Happiness Score"].astype(int)
df.head()

One thing we thought was interesting was to see the development of each feature over the past 10 years. It would be interesting to see it for each country, however considering we have over 150 countries this would be far too much. So we looked at it in groupings, first worldwide and then divided up into Continents/Regions. 

One noteworthy thing we did for plotting is that we zoomed out quite far by setting the range of the y-axis as the lowest/heighest value that would make sense for that feature. We did that because we were more interested in the overall trend and less so in fluctuations. For example, the average of the worldwide Happiness Score when plotted naturally, shows  a lot of ups and downs, but this is really just in the range of -0.1 to +0.1 so not too significant. So really, it just did not change over the past 10 years, which is nicely seen when one zooms out to the natural limits for that feature (0 and 10).

In [ ]:
# plot the worldwide trends over the last 10 years for selected features
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(15,20))

df.groupby("year")["Happiness Score"].mean().plot(ax=axes[0,0])
axes[0, 0].set_ylim(0,10)
axes[0, 0].set_ylabel("Happiness Score")

df.groupby("year")["Social support"].mean().plot(ax=axes[0,1])
axes[0, 1].set_ylim(0, 1)
axes[0, 1].set_ylabel("Social Support")

df.groupby("year")["Healthy life expectancy at birth"].mean().plot(ax=axes[1,0])
axes[1, 0].set_ylim(50, 80)
axes[1, 0].set_ylabel("Healthy life expectancy at birth")

df.groupby("year")["Freedom to make life choices"].mean().plot(ax=axes[1,1])
axes[1, 1].set_ylim(0, 1)
axes[1, 1].set_ylabel("Freedom to make life choices")

df.groupby("year")["GDP"].mean().plot(ax=axes[2,0])
axes[2, 0].set_ylim(500, 600)
axes[2, 0].set_ylabel("GDP")

df.groupby("year")["Infant mortality rate"].mean().plot(ax=axes[2,1])
axes[2, 1].set_ylim(0, 35)
axes[2, 1].set_ylabel("Infant mortality rate")

df.groupby("year")["Positive affect"].mean().plot(ax=axes[3,0])
axes[3, 0].set_ylim(0, 1)
axes[3, 0].set_ylabel("Positive affect")

df.groupby("year")["Negative affect"].mean().plot(ax=axes[3,1])
axes[3, 1].set_ylim(0, 1)
axes[3, 1].set_ylabel("Negative affect")

df.groupby("year")["Democratic Quality"].mean().plot(ax=axes[4,0])
axes[4, 0].set_ylim(-2, 2)
axes[4, 0].set_ylabel("Democratic Quality")

df.groupby("year")["Confidence in national government"].mean().plot(ax=axes[4,1])
axes[4, 1].set_ylim(0, 1)
axes[4, 1].set_ylabel("Confidence in national government")

Observing the trends we cam make out some interesting developments. Many of the features, for the worldwide average, simply seemed to have stagnated. There does not seem to be an overall increase/decrease for features such as Democratic Quality, Positive affect or Happiness. However others showed a trend. Infant mortality rate has been steadily decreasing, so it will be interesting to see if that will correlate inversly with happiness score. In contrast, features such as "Healthy life expectancy at birth" and showed "Freedom to make life choices" showed a slight positive trend.

The reason why so many lines show little or no change is likely because we are looking at the worldwide average here. Surely, there must be regional differences so that is what we look at next.

In [ ]:
# read in a dataset that contains a list of countrys annotated with their region
regions = pd.read_csv("raw_data/WHR151617.csv")
regions = regions[["Country", "Region"]]
regions.drop_duplicates(inplace=True)
regions.dropna(inplace=True)

In [ ]:
# work on a copy of the original df to quickly restore it without having to rerun notebook
main_df = df

In [ ]:
# Merge region data with original df giving each country a region
main_df = pd.merge(main_df, regions,  how='left', left_on=['country'], right_on = ["Country"])
main_df.drop("Country", axis=1, inplace=True)
main_df

In [ ]:
# plot the average happiness score by region over the past 10 years as a bar plot
fig, ax = plt.subplots(figsize=(25,10))
main_df.groupby(["Region", "year"])["Happiness Score"].mean().unstack(level=0).plot(kind="bar", ax=ax, width=0.7)
ax.set_ylim(3, 8)
ax.set_ylabel("Happiness Score")
ax.set_title("Trend of Happiness Score by Region")
ax.legend(bbox_to_anchor=(1,1))

In [ ]:
# plot the average happiness score by region over the past 10 years as a line plot
fig, ax = plt.subplots(figsize=(25,10))
main_df.groupby(["Region", "year"])["Happiness Score"].mean().unstack(level=0).plot(kind="line", ax=ax)
ax.set_ylim(3, 8)
ax.set_ylabel("Happiness Score")
ax.set_title("Trend of Happiness Score by Region")
ax.legend(bbox_to_anchor=(1,1))

Looking at the Happiness score as a line and bar plot by region allows one to see and compare the difference between regions as well as the overall trend. And although there is a bit of rearrangement between regions, with some "overtaking" others during the years there is no real difference between 2008 and 2017. Each regions score fluctuates a bit but often ends up at the same score, with some regions increasing it a bit and others dropping of. For example Australia and New Zealand stay consistently at 7. At this point it is worth noting that this does not mean there is no change at all. When we got the values as floats, we grouped them into bins according to the cantril ladder (0-10). So while the score for Australia and New Zealand could have maybe growing continously over the last 10 years (e.g. from 6.7-7) this trend is lost due to the bining.

Finally we show some of the more interesting descriptive values and their trends. This might not be directly important for feature selection, but it helps getting familiar with the data.

In [ ]:
fig, ax = plt.subplots(figsize=(25,10))
main_df.groupby(["Region", "year"])["Infant mortality rate"].mean().unstack(level=0).plot(kind="bar", ax=ax, width=0.7)
#ax.set_ylim(3, 8)
ax.set_ylabel("Infant mortality rate")
ax.set_title("Trend of Infant mortality rate by Region")
ax.legend(bbox_to_anchor=(1,1))

In [ ]:
fig, ax = plt.subplots(figsize=(25,10))
main_df.groupby(["Region", "year"])["GDP"].mean().unstack(level=0).plot(kind="bar", ax=ax, width=0.7)
ax.set_ylabel("GDP")
ax.set_title("Trend of GDP by Region")
ax.legend(bbox_to_anchor=(1,1))

In [ ]:
fig, ax = plt.subplots(figsize=(25,10))
main_df.groupby(["Region", "year"])["Democratic Quality"].mean().unstack(level=0).plot(kind="bar", ax=ax, width=0.7)
ax.set_ylabel("Democratic Quality")
ax.set_title("Trend of Democratic Quality by Region")
ax.legend(bbox_to_anchor=(1,1))

In [ ]:
fig, ax = plt.subplots(figsize=(25,10))
main_df.groupby(["Region", "year"])["Social support"].mean().unstack(level=0).plot(kind="line", ax=ax)
ax.set_ylabel("Social support")
ax.set_title("Trend of Social support by Region")
ax.legend(bbox_to_anchor=(1,1))

In [ ]:
# New DF - Numerical features:
continuous = df.select_dtypes(['int64', 'float64']).columns


In [ ]:
# Randomize the data:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
# Create a training data set (first 70% of rows)
training_size = int(len(df) * 0.7)
df_train = df[:training_size]
print("Training set size (rows):",len(df_train))

In [ ]:
# Create a test data set (remaining 30% of rows)
training_size = int(len(df) * 0.7)
df_test = df[training_size:]
print("Test set:",len(df_test))

In [ ]:
# A copy of df for full cross validation purposes
df_cross = df

#### Correlations between all continuous features (Continuous vs continuous):

In [ ]:
sns.set(style="white")
# Calculate correlation of all pairs of continuous features
corr = df_train[continuous].corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(30, 30))

# Generate a custom colormap - blue and red
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, annot=True, mask=mask, cmap=cmap, vmax=1, vmin=-1,
            square=True, xticklabels=True, yticklabels=True,
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)
plt.yticks(rotation = 0)
plt.xticks(rotation = 45)

The heatmap above shows the correlation between all continuous features. This is useful for understanding which features affect other features. It is also useful for picking descriptive features to trian a model.

In the context of our dataset so far this means that all of our features are in the heatmap because all of our features are currently continuous.

The following features in df_train have a correlation of 80% or above:

- Delivery quality - Democratic quality
- Life expectancy age 60% - Life expectancy birth


- Neonatal mortality rate - Infant mortality rates
- Under-five mortality rate - Infant mortality rates


- Under-five mortality rate - Neonatal mortality rate


- Life expectancy at birth - Healthy life expectancy at birth
- Life expectancy at age 60 - Healthy life expectancy at birth

The following features in df_train have a correlation of -80% or below:
- Infant mortality rate - Healthy life expectancy at birth
- Neontal mortality rate - Healthy life expectancy at birth
- Under-five mortality rate - Healthy life expectancy at birth


- Infant mortality rate - Life expectancy at birth
- Neontal mortality rate - Life expectancy at birth
- Under-five mortality rate - Life expectancy at birth


Continuous vs categorical features:

Having looked at the heat map, we decided to further examine the correlations between the continuous features and the target feature. Having plotted the scatterplots and reflected upon which features we wanted to include as descriptive features in our model, we chose:
- Social support
- Healthy life expectancy at birth
- Democratic Quality, Delivery Quality
- Life expectancy age 60
- Infant mortality rate
- Positive affect

Their scatter plots are below:

In [ ]:
fig, axs = plt.subplots(2,3, sharey=True)

df_train.plot(kind='scatter', y='Happiness Score', x='Social support', label="%.3f" % df_train[['Social support', 'Happiness Score']].corr().as_matrix()[0,1], ax=axs[0,0], figsize=(15, 6))
df_train.plot(kind='scatter', y='Happiness Score', x='Healthy life expectancy at birth', label="%.3f" % df_train[['Healthy life expectancy at birth', 'Happiness Score']].corr().as_matrix()[0,1], ax=axs[0,1], figsize=(15, 6))
df_train.plot(kind='scatter', y='Happiness Score', x='Democratic Quality', label="%.3f" % df_train[['Democratic Quality', 'Happiness Score']].corr().as_matrix()[0,1], ax=axs[1,0], figsize=(15, 6))
df_train.plot(kind='scatter', y='Happiness Score', x='Delivery Quality', label="%.3f" % df_train[['Delivery Quality', 'Happiness Score']].corr().as_matrix()[0,1], ax=axs[1,1], figsize=(15, 6))
df_train.plot(kind='scatter', y='Happiness Score', x='Life expectancy age 60', label="%.3f" % df_train[['Life expectancy age 60', 'Happiness Score']].corr().as_matrix()[0,1], ax=axs[1,2], figsize=(15, 6))
df_train.plot(kind='scatter', y='Happiness Score', x='Infant mortality rate', label="%.3f" % df_train[['Infant mortality rate', 'Happiness Score']].corr().as_matrix()[0,1], ax=axs[0,2], figsize=(15, 6))


df_train.plot(kind='scatter', y='Happiness Score', x='Positive affect', label="%.3f" % df_train[['Positive affect', 'Happiness Score']].corr().as_matrix()[0,1], figsize=(15, 2))

The correlation plots for the other continuous features are below:

In [ ]:
fig, axs = plt.subplots(2,3, sharey=True)

df_train.plot(kind='scatter', y='Happiness Score', x='Freedom to make life choices', label="%.3f" % df_train[['Freedom to make life choices', 'Happiness Score']].corr().as_matrix()[0,1], ax=axs[0,0], figsize=(15, 6))
df_train.plot(kind='scatter', y='Happiness Score', x='Perceptions of corruption', label="%.3f" % df_train[['Perceptions of corruption', 'Happiness Score']].corr().as_matrix()[0,1], ax=axs[0,1], figsize=(15, 6))
df_train.plot(kind='scatter', y='Happiness Score', x='Generosity', label="%.3f" % df_train[['Generosity', 'Happiness Score']].corr().as_matrix()[0,1], ax=axs[0,2], figsize=(15, 6))
df_train.plot(kind='scatter', y='Happiness Score', x='Negative affect', label="%.3f" % df_train[['Negative affect', 'Happiness Score']].corr().as_matrix()[0,1], ax=axs[1,0], figsize=(15, 6))
df_train.plot(kind='scatter', y='Happiness Score', x='Confidence in national government', label="%.3f" % df_train[['Confidence in national government', 'Happiness Score']].corr().as_matrix()[0,1], ax=axs[1,1], figsize=(15, 6))
df_train.plot(kind='scatter', y='Happiness Score', x='GDP', label="%.3f" % df_train[['GDP', 'Happiness Score']].corr().as_matrix()[0,1], ax=axs[1,2], figsize=(15, 6))


In [ ]:
df_train.plot(kind='scatter', y='Happiness Score', x='Life expectancy birth', label="%.3f" % df_train[['Confidence in national government', 'Happiness Score']].corr().as_matrix()[0,1], figsize=(8, 2))
df_train.plot(kind='scatter', y='Happiness Score', x='Neonatal mortality rate', label="%.3f" % df_train[['Neonatal mortality rate', 'Happiness Score']].corr().as_matrix()[0,1], figsize=(8, 2))
df_train.plot(kind='scatter', y='Happiness Score', x='Under-five mortality rate', label="%.3f" % df_train[['Under-five mortality rate', 'Happiness Score']].corr().as_matrix()[0,1], figsize=(8, 2))

# Linear Regression

We will not train a linear regression model on the data set to produce a set of weights, one for each feature. Those weights will then be applied to the actual values plus an intercept to predict the Happiness Score.

formula: target_feature = w0 + w1 ∗ feature1 + w2 ∗ feature2 + ... + wn ∗ featuren

### Read data from CSV file.

The cleaned data from ealier has been saved to a csv file which I will now use.

In [150]:
df = pd.read_csv('processed_data/cleaned_merged_happiness_data.csv')

df.head(2)

,country,year,Happiness Score,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Confidence in national government,Democratic Quality,Delivery Quality,GDP,Life expectancy birth,Life expectancy age 60,Infant mortality rate,Neonatal mortality rate,Under-five mortality rate
0,Congo (Brazzaville),2014,4,0.685935,53.826542,0.661638,-0.146381,0.808413,0.595255,0.400229,0.483726,-0.733802,-1.153099,545.398597,71.220369,19.669297,24.254883,13.925473,33.447346
1,Mongolia,2014,5,0.943437,61.685921,0.752354,0.130436,0.908597,0.627492,0.170421,0.337257,0.516673,-0.381374,12.196000,68.400000,16.800000,16.900000,10.700000,19.800000


### Changed column headers to remove spaces/capital letters and abbreviate descriptions

I have decided to make some changes to the column headers to make them easier to work with. All changes can be viewed below:  
country = country  
year = year  
happiness_score = Happiness Score  
social_support = Social support  
healthy_life_exp_birth = Healthy life expectancy at birth  
life_choices = Freedom to make life choices  
generosity = Generosity  
corruption = Perceptions of corruption  
pos_effect = Positive affect  
neg_affect = Negative affect  
confidence_gov = Confidence in national government  
dem_quality = Demoncratic Quality  
gdp = GDP  
life_exp_birth = Life expectancy birth  
life_exp_60 = Life expectancy age 60  
infant_mortality = Infant mortality rate  
neonatal_mortality = Neonatal mortality rate  
u5_mortaility = Under-five mortality rate  

In [151]:
df.columns = ["country", "year", "happiness_score", "social_support", "healthy_life_exp_birth", "life_choices", "generosity", "corruption", "pos_affect", "neg_affect", "confidence_gov", "dem_quality", "delivery_quality", "gdp", "life_exp_birth", "life_exp_60", "infant_mortality", "neonatal_mortality", "u5_mortality"]

### Transform happiness_score to a binary class & insert as happiness_class

We were initially going to predict a happiness_score between 0-10 but later decided to predict a binary result with a threshold of 5. I have applied the threshold to the happiness_score to create a new column called happiness_class. A happiness_scores of 5 and above becomes 1, with happiness_scores of less than 5 becoming 0.


In [152]:
happiness_class = (df['happiness_score']>5)*1.0
df_happiness_class = pd.DataFrame({'happiness_class':happiness_class})
df = pd.concat([df, df_happiness_class], axis=1)

In [153]:
df.head(5)

,country,year,happiness_score,social_support,healthy_life_exp_birth,life_choices,generosity,corruption,pos_affect,neg_affect,confidence_gov,dem_quality,delivery_quality,gdp,life_exp_birth,life_exp_60,infant_mortality,neonatal_mortality,u5_mortality,happiness_class
0,Congo (Brazzaville),2014,4,0.685935,53.826542,0.661638,-0.146381,0.808413,0.595255,0.400229,0.483726,-0.733802,-1.153099,545.398597,71.220369,19.669297,24.254883,13.925473,33.447346,0.0
1,Mongolia,2014,5,0.943437,61.685921,0.752354,0.130436,0.908597,0.627492,0.170421,0.337257,0.516673,-0.381374,12.196000,68.400000,16.800000,16.900000,10.700000,19.800000,0.0
2,Costa Rica,2009,8,0.899782,68.641197,0.886061,0.065729,0.786559,0.876206,0.217024,0.521350,0.796451,0.517226,30.826000,79.200000,23.700000,8.900000,6.700000,10.300000,1.0
3,Romania,2017,6,0.811240,66.857086,0.838587,-0.164421,0.925658,0.733730,0.230836,0.298341,0.307430,0.090619,211.315000,74.228571,19.700000,8.775000,4.937500,10.350000,1.0
4,Czech Republic,2014,6,0.877915,69.729263,0.800421,-0.175156,0.896881,0.678407,0.235221,0.339733,1.008329,0.885882,207.818000,71.220369,19.669297,24.254883,13.925473,33.447346,1.0


In [154]:
# mean target feature score
df.happiness_class.mean()

0.453125

In [155]:
df.dtypes

country                    object
year                        int64
happiness_score             int64
social_support            float64
healthy_life_exp_birth    float64
life_choices              float64
generosity                float64
corruption                float64
pos_affect                float64
neg_affect                float64
confidence_gov            float64
dem_quality               float64
delivery_quality          float64
gdp                       float64
life_exp_birth            float64
life_exp_60               float64
infant_mortality          float64
neonatal_mortality        float64
u5_mortality              float64
happiness_class           float64
dtype: object

In [156]:
# check correlation for feature selection
df[[ "happiness_class", "country", "year", "social_support", "healthy_life_exp_birth", "life_choices", "generosity", "corruption", "pos_affect", "neg_affect", "confidence_gov", "dem_quality", "delivery_quality", "gdp", "life_exp_birth", "life_exp_60", "infant_mortality", "neonatal_mortality", "u5_mortality"]].corr()

,happiness_class,year,social_support,healthy_life_exp_birth,life_choices,generosity,corruption,pos_affect,neg_affect,confidence_gov,dem_quality,delivery_quality,gdp,life_exp_birth,life_exp_60,infant_mortality,neonatal_mortality,u5_mortality
happiness_class,1.000000,0.054499,0.601155,0.610530,0.449123,0.128654,-0.315596,0.485218,-0.206525,-0.124023,0.509799,0.559981,0.170105,0.585935,0.631818,-0.530871,-0.546274,-0.507856
year,0.054499,1.000000,-0.001686,0.085714,0.181780,-0.003323,-0.052797,0.003580,0.192507,-0.002864,0.019568,0.005982,0.005750,0.045016,0.040693,-0.066540,-0.061648,-0.068502
social_support,0.601155,-0.001686,1.000000,0.589475,0.415969,0.084150,-0.222379,0.456244,-0.365476,-0.149647,0.541915,0.549914,0.157553,0.570871,0.559523,-0.605337,-0.629924,-0.579524
healthy_life_exp_birth,0.610530,0.085714,0.589475,1.000000,0.335576,0.054289,-0.314577,0.302134,-0.120397,-0.199992,0.621891,0.736790,0.232309,0.918687,0.827421,-0.875910,-0.834034,-0.866297
life_choices,0.449123,0.181780,0.415969,0.335576,1.000000,0.351494,-0.495258,0.623822,-0.290106,0.418441,0.419629,0.458270,0.142719,0.328524,0.367235,-0.290917,-0.307664,-0.285404
generosity,0.128654,-0.003323,0.084150,0.054289,0.351494,1.000000,-0.291499,0.366240,-0.097922,0.276602,0.115362,0.196835,0.048086,0.030423,0.086943,0.040542,0.043253,0.038230
corruption,-0.315596,-0.052797,-0.222379,-0.314577,-0.495258,-0.291499,1.000000,-0.294326,0.248384,-0.449852,-0.285157,-0.498382,-0.077981,-0.307606,-0.348312,0.222328,0.230193,0.205088
pos_affect,0.485218,0.003580,0.456244,0.302134,0.623822,0.366240,-0.294326,1.000000,-0.385255,0.161131,0.378571,0.366900,0.195737,0.304713,0.400902,-0.261728,-0.255344,-0.255688
neg_affect,-0.206525,0.192507,-0.365476,-0.120397,-0.290106,-0.097922,0.248384,-0.385255,1.000000,-0.167158,-0.238025,-0.249651,-0.098236,-0.085052,-0.093485,0.086652,0.082621,0.074034
confidence_gov,-0.124023,-0.002864,-0.149647,-0.199992,0.418441,0.276602,-0.449852,0.161131,-0.167158,1.000000,-0.165711,-0.074368,-0.077217,-0.187935,-0.192838,0.215414,0.218006,0.184351


### Remove a single feature from feature pairs with over 90% correlation

I have decided to remove feature pairs with over 90% correlation,  they are essentially telling us the same thing. The feature from the pair with the lowest level of negative/positive correlation with the target feature will be dropped. See the list of dropped features below:  

Feature correlation: 0.93 life_exp_birth vs healthy_life_exp_birth - drop life_exp_birth  
Feature correlation: 0.99 u5_mortality vs infant_mortality - drop u5_mortality  
Feature correlation: 0.96 neonatal_mortality vs infant_mortality - drop neonatal_mortality  
Feature correlation -0.92 u5_mortality vs life_exp_birth: both features dropped already due to correlation with other descriptive features.


In [157]:
df.drop('life_exp_birth', axis=1, inplace=True)

In [158]:
df.drop('u5_mortality', axis=1, inplace=True)

In [159]:
df.drop('neonatal_mortality', axis=1, inplace=True)

In [160]:
df.head(3)

,country,year,happiness_score,social_support,healthy_life_exp_birth,life_choices,generosity,corruption,pos_affect,neg_affect,confidence_gov,dem_quality,delivery_quality,gdp,life_exp_60,infant_mortality,happiness_class
0,Congo (Brazzaville),2014,4,0.685935,53.826542,0.661638,-0.146381,0.808413,0.595255,0.400229,0.483726,-0.733802,-1.153099,545.398597,19.669297,24.254883,0.0
1,Mongolia,2014,5,0.943437,61.685921,0.752354,0.130436,0.908597,0.627492,0.170421,0.337257,0.516673,-0.381374,12.196000,16.800000,16.900000,0.0
2,Costa Rica,2009,8,0.899782,68.641197,0.886061,0.065729,0.786559,0.876206,0.217024,0.521350,0.796451,0.517226,30.826000,23.700000,8.900000,1.0


### Feature Selection

I have decided to select features with a positive/negative correlation of greater than 0.50 away from 0. This threshold is an arbitrary one that we can change at a later stage if unhappy with the results it yields.  

##### Selected Features:  
social_support (target corr: 0.677275)  
healthy_life_exp_birth (target corr: 0.693580)  
pos_affect (target corr: 0.532431)  
dem_quality (target corr: 0.594980)  
delivery_quality (target corr: 0.670657)  
life_exp_60 (target corr: 0.698286)  
infant_mortality (target corr: -0.595494  

In [161]:
# selected feature correlation
df[["happiness_class", "social_support", "healthy_life_exp_birth", "pos_affect", "dem_quality", "delivery_quality", "life_exp_60", "infant_mortality"]].corr()

,happiness_class,social_support,healthy_life_exp_birth,pos_affect,dem_quality,delivery_quality,life_exp_60,infant_mortality
happiness_class,1.000000,0.601155,0.610530,0.485218,0.509799,0.559981,0.631818,-0.530871
social_support,0.601155,1.000000,0.589475,0.456244,0.541915,0.549914,0.559523,-0.605337
healthy_life_exp_birth,0.610530,0.589475,1.000000,0.302134,0.621891,0.736790,0.827421,-0.875910
pos_affect,0.485218,0.456244,0.302134,1.000000,0.378571,0.366900,0.400902,-0.261728
dem_quality,0.509799,0.541915,0.621891,0.378571,1.000000,0.866750,0.635915,-0.559324
delivery_quality,0.559981,0.549914,0.736790,0.366900,0.866750,1.000000,0.710327,-0.641283
life_exp_60,0.631818,0.559523,0.827421,0.400902,0.635915,0.710327,1.000000,-0.784651
infant_mortality,-0.530871,-0.605337,-0.875910,-0.261728,-0.559324,-0.641283,-0.784651,1.000000


### Training the model

This section trains the model on a linear relationship between descriptive features and the target feature. The data set is split 70/30 into a training set and test set. The purpose of this is to fit the model to the training set and then test that model on the testing set. This process helps avoid over/under fitting a model by using 100% of the data during fitting.


In [162]:
# Create a training data set (first 70% of rows)
training_size = int(len(df) * 0.7)
df_train = df[:training_size]
print("Training set size (rows):",len(df_train))

Training set size (rows): 940


In [163]:
# Create a test data set (remaining 30% of rows)
training_size = int(len(df) * 0.7)
df_test = df[training_size:]
print("Test set:",len(df_test))

Test set: 404


In [164]:
# A copy of df for full cross validation purposes
df_cross = df

In [165]:
# train model on all continuous features using df_train
lm = sm.ols(formula="happiness_class ~ social_support + healthy_life_exp_birth + pos_affect + dem_quality + delivery_quality + life_exp_60 + infant_mortality", data=df_train).fit()
print(lm.params)

Intercept                -3.496194
social_support            1.146434
healthy_life_exp_birth    0.018826
pos_affect                0.837363
dem_quality               0.003198
delivery_quality          0.016873
life_exp_60               0.057482
infant_mortality          0.005188
dtype: float64


### Table with feature weights

The model has been trained on the df_train data set. A table helps us understand how some of the features performed.

In [166]:
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:        happiness_class   R-squared:                       0.559
Model:                            OLS   Adj. R-squared:                  0.555
Method:                 Least Squares   F-statistic:                     168.4
Date:                Sun, 29 Apr 2018   Prob (F-statistic):          1.44e-160
Time:                        15:07:14   Log-Likelihood:                -293.25
No. Observations:                 939   AIC:                             602.5
Df Residuals:                     931   BIC:                             641.3
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -3

### Test the model on df_train

Note: we also evaluate the model on the df_test to avoid overfitting

In [167]:
lm.predict(df_train)

0      0.036646
1      0.320742
2      0.981082
3      0.487344
4      0.665732
5      0.476788
6     -0.055179
7      1.093609
8      0.053434
9      0.104186
10     0.367144
11     0.502436
12     0.491271
13    -0.003944
14     0.235502
15     0.020736
16     0.371749
17     0.856565
18     0.352984
19     0.903477
20     0.315044
21     0.087298
22     0.185864
23     0.136093
24     0.110452
25     0.398386
26    -0.060803
27     0.195335
28     0.769744
29     1.007414
         ...   
910    1.096086
911    0.689729
912    1.108037
913    0.350294
914    0.646326
915    0.732949
916    1.036209
917    0.036776
918    0.328240
919    0.266324
920    0.925225
921    0.694990
922    0.746642
923    0.414688
924    0.909340
925    0.384983
926    0.508478
927    0.381444
928    0.164294
929    0.671003
930    0.135603
931    0.020256
932    0.447776
933    0.446043
934    0.359670
935    0.400937
936    0.500616
937    0.305471
938    0.507283
939    0.498192
Length: 940, dtype: floa

### Actual Happiness Score vs Predicted Happiness

In [168]:
predict_df_train = pd.DataFrame({'ActualHappiness': df_train.happiness_class, 'PredictedHappiness': lm.predict(df_train)})
predict_df_train.head(10)

,ActualHappiness,PredictedHappiness
0,0.0,0.036646
1,0.0,0.320742
2,1.0,0.981082
3,1.0,0.487344
4,1.0,0.665732
5,0.0,0.476788
6,0.0,-0.055179
7,1.0,1.093609
8,0.0,0.053434
9,0.0,0.104186


### Actual Happiness Score minus Predicted Score

In [169]:
# analyse the value of the actual happiness score minus the predicted happiness score
print("Actual minus Predicted:\n", (df_train.happiness_class - lm.predict(df_train)))
print("\n(Actual minus Predicted) squared:\n", (df_train.happiness_class - lm.predict(df_train))**2)

Actual minus Predicted:
 0     -0.036646
1     -0.320742
2      0.018918
3      0.512656
4      0.334268
5     -0.476788
6      0.055179
7     -0.093609
8     -0.053434
9     -0.104186
10     0.632856
11    -0.502436
12     0.508729
13     0.003944
14    -0.235502
15    -0.020736
16     0.628251
17     0.143435
18    -0.352984
19     0.096523
20    -0.315044
21    -0.087298
22    -0.185864
23    -0.136093
24    -0.110452
25    -0.398386
26     0.060803
27    -0.195335
28     0.230256
29    -0.007414
         ...   
910   -0.096086
911    0.310271
912   -0.108037
913   -0.350294
914    0.353674
915    0.267051
916   -0.036209
917   -0.036776
918   -0.328240
919   -0.266324
920    0.074775
921   -0.694990
922    0.253358
923    0.585312
924    0.090660
925   -0.384983
926   -0.508478
927   -0.381444
928   -0.164294
929    0.328997
930   -0.135603
931   -0.020256
932    0.552224
933   -0.446043
934   -0.359670
935    0.599063
936    0.499384
937   -0.305471
938    0.492717
939   -0.498192

# This is where we need to place evaluation metrics

I have tried to get the evaluation metrics below to work on the linear model but I can't seem to do this. They include the accuracy, confusion matrix and classification report which i have taken from the logistic regression practical. I would appreciate it if someone could have a quick look please as I can't spend anymore time at it without some guidance.

In [170]:
# # Some more evaluation metrics.
# print("Accuracy: ", metrics.accuracy_score(df_train['happiness_class'], predictions))
# print("Confusion matrix: \n", metrics.confusion_matrix(df_train['happiness_class'], predictions))
# print("Classification report:\n ", metrics.classification_report(df_train['happiness_class'], predictions))

### Test on df_test

In [171]:
predict_df_test = pd.DataFrame({'ActualHappiness': df_test.happiness_class, 'PredictedHappiness': lm.predict(df_test)})
predict_df_test.head(10)

,ActualHappiness,PredictedHappiness
940,1.0,0.963398
941,0.0,0.484414
942,1.0,0.826590
943,0.0,0.063579
944,1.0,0.842991
945,0.0,0.744982
946,1.0,0.848885
947,1.0,0.884778
948,0.0,0.834724
949,1.0,0.122037


# Perform evaluation metrics on df_test data set results

In [173]:
# # Some more evaluation metrics.
# print("Accuracy: ", metrics.accuracy_score(df_train['happiness_class'], predictions))
# print("Confusion matrix: \n", metrics.confusion_matrix(df_train['happiness_class'], predictions))
# print("Classification report:\n ", metrics.classification_report(df_train['happiness_class'], predictions))

### Perfrom normalisation of the features

This process will give each feature a relative value and help to compare the coef of different features.

In [174]:
df_feat = df_train[['happiness_class', 'social_support', 'healthy_life_exp_birth', 'pos_affect', 'dem_quality', 'delivery_quality', 'life_exp_60', 'infant_mortality']]

In [175]:
df_feat.min()

happiness_class            0.000000
social_support             0.290934
healthy_life_exp_birth    39.351990
pos_affect                 0.362498
dem_quality               -2.315310
delivery_quality          -2.144974
life_exp_60               12.600000
infant_mortality           1.700000
dtype: float64

In [176]:
df_feat.max()

happiness_class             1.000000
social_support              0.987343
healthy_life_exp_birth     76.268028
pos_affect                  0.943621
dem_quality                 1.540097
delivery_quality            2.121312
life_exp_60                26.000000
infant_mortality          116.200000
dtype: float64

In [177]:
# range normalise all columns
df_norm = (df_feat - df_feat.min()) / (df_feat.max() - df_feat.min())
df_norm.head(10)

,happiness_class,social_support,healthy_life_exp_birth,pos_affect,dem_quality,delivery_quality,life_exp_60,infant_mortality
0,0.0,0.567196,0.392094,0.400530,0.410205,0.232491,0.527559,0.196986
1,0.0,0.936953,0.604993,0.456003,0.734548,0.413381,0.313433,0.132751
2,1.0,0.874267,0.793401,0.883992,0.807116,0.624009,0.828358,0.062882
3,1.0,0.747127,0.745072,0.638820,0.680276,0.524014,0.529851,0.061790
4,1.0,0.842868,0.822875,0.543619,0.862072,0.710420,0.527559,0.196986
5,0.0,0.868397,0.702959,0.504745,0.805441,0.668555,0.522388,0.029694
6,0.0,0.650238,0.295347,0.313795,0.486657,0.405585,0.201493,0.515284
7,1.0,0.930162,0.853349,0.908107,0.918191,0.927821,0.843284,0.028821
8,0.0,0.464510,0.698775,0.272604,0.484728,0.478622,0.507463,0.103930
9,0.0,0.612352,0.718619,0.100606,0.480294,0.479184,0.511727,0.102838


### Train Linear Model on normalised columns

The values of each column have been normalised so we can now train a model on them and compare the coef with one another.

In [180]:
lm_df_norm = sm.ols(formula="happiness_class ~ social_support + healthy_life_exp_birth + pos_affect + dem_quality + delivery_quality + life_exp_60 + infant_mortality", data=df_norm).fit()
print(lm_df_norm.params)

Intercept                -1.428766
social_support            0.798387
healthy_life_exp_birth    0.694992
pos_affect                0.486611
dem_quality               0.012330
delivery_quality          0.071986
life_exp_60               0.770262
infant_mortality          0.594016
dtype: float64


In [181]:
print(lm_df_norm.summary())

                            OLS Regression Results                            
Dep. Variable:        happiness_class   R-squared:                       0.559
Model:                            OLS   Adj. R-squared:                  0.555
Method:                 Least Squares   F-statistic:                     168.4
Date:                Sun, 29 Apr 2018   Prob (F-statistic):          1.44e-160
Time:                        15:09:38   Log-Likelihood:                -293.25
No. Observations:                 939   AIC:                             602.5
Df Residuals:                     931   BIC:                             641.3
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

### Standardisation of features

In [183]:
df_st = (df_feat - df_feat.mean()) / df_feat.std()
df_st.head(10)

,happiness_class,social_support,healthy_life_exp_birth,pos_affect,dem_quality,delivery_quality,life_exp_60,infant_mortality
0,-0.909897,-0.992990,-1.076315,-1.034022,-0.663670,-1.145520,-0.002097,-0.024071
1,-0.909897,1.131586,-0.077610,-0.738956,0.770861,-0.363057,-1.020988,-0.362276
2,1.097857,0.771401,0.806209,1.537541,1.091819,0.548045,1.429212,-0.730145
3,1.097857,0.040870,0.579499,0.233454,0.530820,0.115503,0.008806,-0.735893
4,1.097857,0.590987,0.944471,-0.272925,1.334883,0.921832,-0.002097,-0.024071
5,-0.909897,0.737673,0.381947,-0.479697,1.084411,0.740738,-0.026704,-0.904883
6,-0.909897,-0.515842,-1.530154,-1.495367,-0.325534,-0.396779,-1.553640,1.651809
7,1.097857,1.092566,1.087426,1.665811,1.583089,1.862228,1.500232,-0.909481
8,-0.909897,-1.583014,0.362321,-1.714466,-0.334062,-0.080847,-0.097724,-0.514022
9,-0.909897,-0.733530,0.455410,-2.629335,-0.353673,-0.078416,-0.077433,-0.519770


In [184]:
lm_df_st = sm.ols(formula="happiness_class ~ social_support + healthy_life_exp_birth + pos_affect + dem_quality + delivery_quality + life_exp_60 + infant_mortality", data=df_st).fit()
print(lm_df_st.summary())

                            OLS Regression Results                            
Dep. Variable:        happiness_class   R-squared:                       0.559
Model:                            OLS   Adj. R-squared:                  0.555
Method:                 Least Squares   F-statistic:                     168.4
Date:                Sun, 29 Apr 2018   Prob (F-statistic):          1.44e-160
Time:                        15:09:58   Log-Likelihood:                -947.75
No. Observations:                 939   AIC:                             1911.
Df Residuals:                     931   BIC:                             1950.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0